<a href="https://colab.research.google.com/github/Nanditha587/Traffic-signal-recognition/blob/main/Another_copy_of_CNN_Traffic_Signs_Final_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

print("Drive mounted.")

Mounted at /content/gdrive
Drive mounted.


In [ ]:

ZIP_FILE_PATH = '/content/gdrive/MyDrive/traffic signals.zip'
UNZIP_DESTINATION = '/content/GTSRB_Data/'
!mkdir -p "$UNZIP_DESTINATION"
!unzip -q "$ZIP_FILE_PATH" -d "$UNZIP_DESTINATION"

print("\n--- Unzipping Started ---")
print(f"Images will be extracted to: {UNZIP_DESTINATION}. Please wait for the cell to finish executing.")


--- Unzipping Started ---
Images will be extracted to: /content/GTSRB_Data/. Please wait for the cell to finish executing.


In [ ]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from glob import glob
import os

In [ ]:

train_dir = '/content/GTSRB_Data/Train/'
NUM_CLASSES = 43
IMG_SIZE = 30
try:
    class_folders = os.listdir(train_dir)
    print(f"Found {len(class_folders)} subdirectories (classes) in the training folder.")
except FileNotFoundError:
    print(f"Error: Could not find the directory at {train_dir}. Check your unzipping location.")

Found 43 subdirectories (classes) in the training folder.


In [ ]:

!ls -F /content/GTSRB_Data/Train/

0/   11/  14/  17/  2/	 22/  25/  28/	30/  33/  36/  39/  41/  6/  9/
1/   12/  15/  18/  20/  23/  26/  29/	31/  34/  37/  4/   42/  7/
10/  13/  16/  19/  21/  24/  27/  3/	32/  35/  38/  40/  5/	 8/


In [ ]:

train_dir = '/content/GTSRB_Data/Train/'
X_data = []
y_labels = []

NUM_CLASSES = 43
IMG_SIZE = 30

print("Starting image loading.")


for i in range(NUM_CLASSES):
    folder_name = str(i)
    path = os.path.join(train_dir, folder_name)

    if not os.path.isdir(path):

        continue

    images = os.listdir(path)


    for img in images:
        if not img.endswith(('.png', '.ppm', '.jpg')):
            continue

        try:
            image_path = os.path.join(path, img)
            image = cv2.imread(image_path)

            if image is None:
                continue

            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            X_data.append(np.array(image))
            y_labels.append(i)

        except Exception as e:
            continue

print("Image loading complete.")
print(f"Total images loaded: {len(X_data)}")

Starting image loading.
Image loading complete.
Total images loaded: 0


In [ ]:
from sklearn.model_selection import train_test_split


X = np.array(X_data)
y = np.array(y_labels)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_val = X_val / 255.0


y_train_encoded = to_categorical(y_train, num_classes=NUM_CLASSES)
y_val_encoded = to_categorical(y_val, num_classes=NUM_CLASSES)

print("Final Preprocessing Complete.")
print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:


model = Sequential([

    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),


    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),


    Flatten(),


    Dense(128, activation='relu'),
    Dropout(0.5), Dense(NUM_CLASSES, activation='softmax')
])


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("\nBaseline CNN Model Architecture Defined and Compiled.")
model.summary()


In [ ]:

history = model.fit(X_train, y_train_encoded,
                    epochs=15,
                    batch_size=32,
                    validation_data=(X_val, y_val_encoded),
                    verbose=1)

print("\nModel training started. Wait for 15 epochs to complete.")

In [ ]:

loss, accuracy = model.evaluate(X_val, y_val_encoded, verbose=0)
print(f"\nFinal Validation Accuracy: {accuracy*100:.2f}%")
print(f"Final Validation Loss: {loss:.4f}")

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:

TEST_IMAGE_PATH = '/content/test_u_turn.png'
IMG_SIZE = 30
TRUE_LABEL = 33
try:
    image = cv2.imread(TEST_IMAGE_PATH)

    if image is None:
        print(f"Error: Image not found at {TEST_IMAGE_PATH}. Please check the filename.")
    else:

        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))


        normalized_image = image / 255.0


        input_data = np.expand_dims(normalized_image, axis=0)

        prediction_probs = model.predict(input_data)
        predicted_class = np.argmax(prediction_probs)
        confidence = np.max(prediction_probs) * 100

        plt.figure(figsize=(4, 4))


        display_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(display_image)
        plt.title(f"Predicted Class: {predicted_class} (Conf: {confidence:.2f}%)")
        plt.axis('off')
        plt.show()

        print(f"\n--- INFERENCE RESULT ---")
        print(f"True Class (U-Turn): {TRUE_LABEL}")
        print(f"Model Predicted Class: {predicted_class}")
        print(f"Confidence: {confidence:.2f}%")

except Exception as e:
    print(f"An error occurred during inference: {e}")

In [ ]:

!mkdir -p '/content/gdrive/MyDrive/Traffic_Sign_Model/'


model.save('/content/gdrive/MyDrive/Traffic_Sign_Model/baseline_cnn_gtsrb.h5')

print("Model successfully saved to your Google Drive.")

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from glob import glob
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier # For k-NN

In [ ]:

print("Re-unzipping data now to ensure files exist at the local path.")

ZIP_FILE_PATH = '/content/gdrive/MyDrive/traffic signals.zip'


UNZIP_DESTINATION = '/content/GTSRB_Data/'

!mkdir -p "$UNZIP_DESTINATION"
!unzip -q "$ZIP_FILE_PATH" -d "$UNZIP_DESTINATION"

print("\n--- Re-unzip complete. ---")

In [ ]:

train_dir = '/content/GTSRB_Data/Train/'
NUM_CLASSES = 43
IMG_SIZE = 30
X_data = []
y_labels = []

print("Starting image loading...")

for i in range(NUM_CLASSES):
    folder_name = str(i)
    path = os.path.join(train_dir, folder_name)

    if not os.path.isdir(path):
        continue

    images = os.listdir(path)

    for img in images:
        if not img.endswith(('.png', '.ppm', '.jpg')):
            continue

        try:
            image_path = os.path.join(path, img)
            image = cv2.imread(image_path)

            if image is None:
                continue

            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            X_data.append(np.array(image))
            y_labels.append(i)

        except Exception as e:
            continue

print(f"Total images loaded: {len(X_data)}")


X = np.array(X_data)
y = np.array(y_labels)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


X_train = X_train / 255.0
X_val = X_val / 255.0
print("Data ready for feature extraction.")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model


NUM_CLASSES = 43
IMG_SIZE = 30
MODEL_PATH = '/content/gdrive/MyDrive/Traffic_Sign_Model/baseline_cnn_gtsrb.h5'


try:

    model = load_model(MODEL_PATH)


    dummy_input = tf.zeros((1, IMG_SIZE, IMG_SIZE, 3))
    _ = model(dummy_input)
    print("Model loaded and built by dummy input pass.")

except Exception as e:
    print(f"FATAL: Error loading model: {e}")



feature_layer = model.layers[-2].output


feature_function = Model(inputs=model.layers[0].input, outputs=feature_layer)



X_train_features = feature_function.predict(X_train, verbose=0)
X_val_features = feature_function.predict(X_val, verbose=0)

print(f"Extracted feature shape (Validation Features): {X_val_features.shape}")

In [ ]:


from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train_features, y_train)


y_pred_knn = knn_classifier.predict(X_val_features)
knn_accuracy = accuracy_score(y_val, y_pred_knn)

print("k-NN Training Complete.")
print(f"k-NN (on CNN Features) Accuracy: {knn_accuracy*100:.2f}%")

In [ ]:


from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

print("\n--- Starting SVM Comparison ---")
print("NOTE: SVM training is computationally intensive and may take a significant amount of time.")


svm_classifier = SVC(kernel='rbf', C=10, gamma='auto', random_state=42)

svm_classifier.fit(X_train_features, y_train)

print("SVM Training Complete.")


y_pred_svm = svm_classifier.predict(X_val_features)
svm_accuracy = accuracy_score(y_val, y_pred_svm)

print(f"SVM (on CNN Features) Accuracy: {svm_accuracy*100:.2f}%")